# GWAS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QAHRoddur/JWAS/blob/main/Examples/gwas.ipynb)

This notebook is auto-generated from the JWAS.jl wiki page.


In [ ]:
using Pkg
Pkg.add("JWAS")
Pkg.precompile()
using JWAS


# GWAS

Run Bayesian analysis at first.


In [ ]:
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets

# Step 2: Read data 
phenofile  = "../data/phenotypes.txt"
pedfile    = "../data/pedigree.txt"
genofile   = "../data/genotypes.txt"
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstring=["NA"])
pedigree   = get_pedigree(pedfile,separator=",",header=true);
genotypes  = get_genotypes(genofile,separator=',',method="BayesC");
first(phenotypes,5)

# Step 3: Build Model Equations

model_equation  ="y1 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes
                  y2 = intercept + x1 + x2 + ID + genotypes
                  y3 = intercept + x1 + ID + genotypes";
model = build_model(model_equation);

# Step 4: Set Factors or Covariates
set_covariate(model,"x1");

# Step 5: Set Random or Fixed Effects
set_random(model,"x2");
set_random(model,"ID dam",pedigree);

# Step 6: Run Analysis
out=runMCMC(model,phenotypes);


After running the analysis above, GWAS can be performed as follows.

**1. Compute the model frequency for each marker (the probability the marker is included in the model).**


In [ ]:
marker_effects_file="results/MCMC_samples_marker_effects_genotypes_y1.txt"
GWAS(marker_effects_file,header=true)


**2. Compute the posterior probability of association of the genomic window that explains a large proportion of the total genetic variance.**


In [ ]:
map_file="../data/map.txt"
marker_effects_file="results/MCMC_samples_marker_effects_genotypes_y1.txt"
out=GWAS(model,map_file,marker_effects_file,header=true,window_size="1 Mb")


**3. Compute the genetic correlation between two correlated traits for each genomic window.**


In [ ]:
map_file="../data/map.txt"
marker_effects_file1 ="results/MCMC_samples_marker_effects_genotypes_y1.txt"
marker_effects_file2 ="results/MCMC_samples_marker_effects_genotypes_y2.txt"
out=GWAS(model,map_file,marker_effects_file1,marker_effects_file2,genetic_correlation=true,header=true,window_size="1 Mb")
